## ETL Pipeline - Olist dataset

### Connect Python to SQL Server

In [22]:
import pyodbc


server = 'localhost'
database = 'OlistDataset'
username = 'sa'
password = 'pokita'

connection_string = f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}'

try:
    connection = pyodbc.connect(connection_string)
    print("Connection successful!")
    
except pyodbc.Error as e:
    print(f"Error: {e}")
    
finally:
    if 'connection' in locals() and connection:
        connection.close()


Connection successful!


### Load all CSV files into a Jupyter Notebook using Pandas.

In [24]:
import pandas as pd

base_path = r'C:\Users\use\Documents\Python Scripts\archive'

orders = pd.read_csv(f'{base_path}\\olist_orders_dataset.csv')
customers = pd.read_csv(f'{base_path}\\olist_customers_dataset.csv')
products = pd.read_csv(f'{base_path}\\olist_products_dataset.csv')
sellers = pd.read_csv(f'{base_path}\\olist_sellers_dataset.csv')
order_items = pd.read_csv(f'{base_path}\\olist_order_items_dataset.csv')
order_payments = pd.read_csv(f'{base_path}\\olist_order_payments_dataset.csv')
order_reviews = pd.read_csv(f'{base_path}\\olist_order_reviews_dataset.csv')



In [25]:
pd.set_option('display.max_columns', None) 
pd.set_option('display.max_rows', 10)     
pd.set_option('display.width', 1000)      

order_reviews.head()

review_id                          order_id  review_score review_comment_title                             review_comment_message review_creation_date review_answer_timestamp
0  7bc2406110b926393aa56f80a40eba40  73fc7af87114b39712e6da79b0a377eb             4                  NaN                                                NaN  2018-01-18 00:00:00     2018-01-18 21:46:59
1  80e641a11e56f04c1ad469d5645fdfde  a548910a1c6147796b98fdf73dbeba33             5                  NaN                                                NaN  2018-03-10 00:00:00     2018-03-11 03:05:13
2  228ce5500dc1d8e020d8d1322874b6f0  f9e4b658b201a9f2ecdecbb34bed034b             5                  NaN                                                NaN  2018-02-17 00:00:00     2018-02-18 14:36:24
3  e64fb393e7b32834bb789ff8bb30750e  658677c97b385a9be170737859d3511b             5                  NaN              Recebi bem antes do prazo estipulado.  2017-04-21 00:00:00     2017-04-21 22:02:06
4  f7c4243c7fe1938f181bec41a392bdeb  8e6bfb81e283fa7e4f11123a3fb894f1             5                  NaN  Parabéns lojas lannister adorei comprar pela I...  2018-03-01 00:00:00     2018-03-02 10:26:53

### Req #1: Clean the datasets

In [27]:
orders.dropna(subset=['order_id'], inplace=True)
customers.dropna(subset=['customer_id'], inplace=True)
products.dropna(subset=['product_id'], inplace=True)
sellers[['seller_zip_code_prefix']] = sellers[['seller_zip_code_prefix']].fillna(value="Unknown")
order_items.dropna(inplace=True)
order_payments.dropna(inplace=True)
order_reviews[['review_comment_title', 'review_comment_message']] = order_reviews[['review_comment_title', 'review_comment_message']].fillna(value="None")


orders.drop_duplicates(inplace=True)
customers.drop_duplicates(inplace=True)
products.drop_duplicates(inplace=True)
sellers.drop_duplicates(inplace=True)
order_items.drop_duplicates(inplace=True)
order_payments.drop_duplicates(inplace=True)
order_reviews.drop_duplicates(inplace=True)

In [28]:
new_duplicate_count = orders.duplicated().sum()
print(f"Number of duplicate rows after dropping: {new_duplicate_count}") 

Number of duplicate rows after dropping: 0


### Req #2: Create Calculated Columns

In [30]:
# converting to datetime
orders['order_purchase_timestamp'] = pd.to_datetime(orders['order_purchase_timestamp'])
orders['order_delivered_customer_date'] = pd.to_datetime(orders['order_delivered_customer_date'])
orders['order_approved_at'] = pd.to_datetime(orders['order_approved_at'])
orders['order_estimated_delivery_date'] = pd.to_datetime(orders['order_estimated_delivery_date'])

# merge of tables needed for calculation (order_items, orders, order_payments)
merged_df = pd.merge(order_items, orders, on='order_id', how='left')
merged_df = pd.merge(merged_df, order_payments[['order_id', 'payment_installments']], on='order_id', how='left')

# total price
merged_df['total_price'] = merged_df['price'] + merged_df['freight_value']

# delivery time
merged_df['delivery_time'] = (merged_df['order_delivered_customer_date'] - merged_df['order_purchase_timestamp']).dt.days

# payment count
merged_df['payment_installments'] = merged_df['payment_installments'].fillna(0)
payment_count = merged_df.groupby('order_id')['payment_installments'].transform('sum')
merged_df['payment_count'] = payment_count

# profit margit
merged_df['profit_margin'] = merged_df['price'] - merged_df['freight_value']
merged_df['payment_count'] = payment_count


# updated dataframe
merged_df[['order_id', 'total_price', 'delivery_time', 'payment_count', 'profit_margin']].head()


order_id  total_price  delivery_time  payment_count  profit_margin
0  00010242fe8c5a6d1ba2dd792cb16214        72.19            7.0            2.0          45.61
1  00018f77f2f0320c557190d7a144bdd3       259.83           16.0            3.0         219.97
2  000229ec398224ef6ca0657da4fc703e       216.87            7.0            5.0         181.13
3  00024acbcdf0a6daa1e931b038114c75        25.78            6.0            2.0           0.20
4  00042b26cf59d7ce69dfabb4e55b4fd9       218.04           25.0            3.0         181.76

### Req #3: Use Window Functions Over Partitions (Pandas)

In [32]:

# merge order_items and orders
merged_df = pd.merge(order_items, orders[['order_id', 'customer_id', 'order_purchase_timestamp', 'order_delivered_customer_date']], on='order_id', how='left')

# merge with products
merged_df = pd.merge(merged_df, products[['product_id', 'product_category_name']], on='product_id', how='left')

# total sales per customer
merged_df['total_sales_per_customer'] = merged_df.groupby('customer_id')['price'].cumsum()

# average delivery time per product category
merged_df['delivery_time'] = (merged_df['order_delivered_customer_date'] - merged_df['order_purchase_timestamp']).dt.days
merged_df['average_delivery_time_per_category'] = merged_df.groupby('product_category_name')['delivery_time'].rolling(window=3, min_periods=1).mean().reset_index(0, drop=True)
  
# updated dataframe
merged_df[['customer_id', 'product_category_name', 'price', 'total_sales_per_customer', 'delivery_time', 'average_delivery_time_per_category']].head()


customer_id product_category_name   price  total_sales_per_customer  delivery_time  average_delivery_time_per_category
0  3ce436f183e68e07877b285a838db11a            cool_stuff   58.90                     58.90            7.0                                 7.0
1  f6dd3ec061db4e3987629fe6b26e5cce              pet_shop  239.90                    239.90           16.0                                16.0
2  6489ae5e4333f3693df5ad4372dab6d3      moveis_decoracao  199.00                    199.00            7.0                                 7.0
3  d4eb9395c8c0431ee92fce09860c5a06            perfumaria   12.99                     12.99            6.0                                 6.0
4  58dbd0b2d70206bf40e62cd34e84d795    ferramentas_jardim  199.90                    199.90           25.0                                25.0

### Req #4: Saving Processed Data to SQL Server (Fact & Dimension Tables)

In [34]:
import pandas as pd
from sqlalchemy import create_engine
import pyodbc

server = 'localhost'
database = 'OlistDataset'
username = 'sa'
password = 'pokita'

connection_string = f'mssql+pyodbc://{username}:{password}@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server'

try:
    engine = create_engine(connection_string)
    connection = engine.connect()
    print("Connection successful!")
    connection.close()
except Exception as e:
    print(f"Connection failed: {e}")


Connection successful!


In [35]:
print(products.columns)


Index(['product_id', 'product_category_name', 'product_name_lenght', 'product_description_lenght', 'product_photos_qty', 'product_weight_g', 'product_length_cm', 'product_height_cm', 'product_width_cm'], dtype='object')


In [36]:
merged_df['total_price'] = merged_df['price'] + merged_df['freight_value']
merged_df['payment_count'] = payment_count
merged_df['profit_margin'] = merged_df['price'] - merged_df['freight_value']

# creation of fact table
fact_table = merged_df[['customer_id','order_id', 'product_id', 'seller_id','price', 'freight_value', 
                        'total_price', 'delivery_time', 'payment_count', 'profit_margin']]

# creation of dimension table for customers
dim_customers = customers[['customer_id', 'customer_zip_code_prefix', 'customer_city', 'customer_state']]

# creation of dimension table for products
dim_products = products[['product_id', 'product_category_name', 'product_name_lenght', 'product_description_lenght',
                         'product_photos_qty', 'product_weight_g', 
                         'product_length_cm', 'product_height_cm', 'product_width_cm']]

# creation of dimension table for sellers
dim_sellers = sellers[['seller_id', 'seller_zip_code_prefix', 'seller_city', 'seller_state']]

# creation of dimension table for dates

dim_date = pd.merge(
    orders[['order_id', 'order_purchase_timestamp', 'order_delivered_customer_date']],
    merged_df[['order_id', 'seller_id']], 
    on='order_id',
    how='left'
)[['order_id', 'seller_id', 'order_purchase_timestamp', 'order_delivered_customer_date']]


# save the tables to SQL Server
fact_table.to_sql('fact_order_items', engine, if_exists='replace', index=False)
dim_customers.to_sql('dim_customers', engine, if_exists='replace', index=False)
dim_products.to_sql('dim_products', engine, if_exists='replace', index=False)
dim_sellers.to_sql('dim_sellers', engine, if_exists='replace', index=False)
dim_date.to_sql('dim_date', engine, if_exists='replace', index=False)

print("Data has been successfully saved to SQL Server!")

Data has been successfully saved to SQL Server!
